In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
import json
from scipy import interpolate
import langmuir.gymness.aggregate as agg
from decimal import Decimal
import pandas as pd
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
# definisco i file
lamp_file = '../OES_Studenti_2017/Ilamp_acquisitiontime_3s.txt'
rho_file = '../OES_Studenti_2017/riflettivitadiffusore.txt' 
argon_750_file = '../OES_Studenti_2017/Ar15022017_a8_acq_time_0.01s.txt' 
argon_480_file = '../OES_Studenti_2017/Ar15022017_a1_acq_time_0.5s.txt' 

acquisition_time = 0.01

peak_ratio = functions.get_peak_ratio(argon_480_file)
[I_abs_spectrum, n_0, I_abs_750, X_750] = functions.get_absolute_intensity(lamp_file, rho_file, argon_750_file, acquisition_time)
n_e_mean, n_e_min, n_e_max = functions.get_density(peak_ratio, '../fantz-data/fants.txt')

In [3]:
# peak_ratio_approx = '%.3E' % peak_ratio
# n_0_approx = '%.3E' % n_0
# I_abs_750_approx = '%.3E' % I_abs_750
# X_750_approx = '%.3E' % X_750
# n_e_mean_approx = '%.3E' % n_e_mean
# n_e_min_approx = '%.3E' % n_e_min
# n_e_max_approx = '%.3E' % n_e_max
print("""
Peak ratio I480/I488 = {:.3E}
n0 = {:.3E} cm-3
I absolute 750 = {:.3E} 
X 750 = {:.3E}
n_e_mean = {:.3E} cm-3
( {:.3E} < n_e < {:.3E} ) cm-3""". format(peak_ratio, n_0, I_abs_750, X_750, n_e_mean, n_e_min, n_e_max))


Peak ratio I480/I488 = 5.252E-01
n0 = 1.204E+18 cm-3
I absolute 750 = 1.986E-13 
X 750 = 4.782E+16
n_e_mean = 1.200E+11 cm-3
( 1.072E+11 < n_e < 1.338E+11 ) cm-3


In [4]:
# density ratio pression spectra analysis
#                                                    name of config file    folder with data
ratios_p1, w_em_p1, results_p1 = functions.get_profile_ratio('../config_p1_2302.json', '../data/2302/2302spectr/')
ratios_p2, w_em_p2, results_p2 = functions.get_profile_ratio('../config_p2_2302.json', '../data/2302/2302spectr/')
n_avg_p1, n_min_p1, n_max_p1  = functions.get_density(ratios_p1, '../fantz-data/fants.txt')
n_avg_p2, n_min_p2, n_max_p2  = functions.get_density(ratios_p2, '../fantz-data/fants.txt')
p1_off = 2.40e-5
p2_off = 1.34e-4

In [5]:
# create a pandas dataframe for densities
df_spectra_n = pd.DataFrame()
# for p1
for w, content in results_p1.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p1_off
    df_spectra_n = pd.concat([df_spectra_n, df])
    
# for p2
for w, content in results_p2.items():
    df = pd.read_json(json.dumps(content), orient="records")
    df['w'] = w
    df['p_off'] = p2_off
    df_spectra_n = pd.concat([df_spectra_n, df])

# apply the get density function to every row and save the results in a density dataframe
df = pd.DataFrame(df_spectra_n['ratio'].apply(functions.get_density).tolist(), columns=['n', 'n_min', 'n_max'])
df_spectra_n = df_spectra_n.reset_index()
del df_spectra_n['index']

# units


# concatenate the two dataframes
df_spectra_n = pd.concat([df_spectra_n, df], axis=1)
# group by power and take average
a = df_spectra_n.groupby(['p_off', 'w', 'id']).mean()
#a

df_spectra_n.columns = ['id', 'p (mbar)', 'ratio', 'w (%)', 'p_off (mbar)',
                       'n ($cm^{-3}$)', 'n_min ($cm^{-3}$)', 'n_max ($cm^{-3}$)']
df_spectra_n

,id,p (mbar),ratio,w (%),p_off (mbar),n ($cm^{-3}$),n_min ($cm^{-3}$),n_max ($cm^{-3}$)
0,a1ad1,0.000000,0.480338,40,0.000024,1.091107e+11,9.585906e+10,1.217129e+11
1,a3ad3,0.000000,0.470204,40,0.000024,1.063948e+11,9.151617e+10,1.189742e+11
2,a7ad7,0.000000,0.471929,40,0.000024,1.068571e+11,9.225542e+10,1.194404e+11
3,a12ad12,0.000039,0.461439,20,0.000024,1.037000e+11,8.775966e+10,1.166052e+11
4,a13ad13,0.000039,0.454845,20,0.000024,1.016664e+11,8.493361e+10,1.148230e+11
5,a14ad14,0.000038,0.452504,20,0.000024,1.009446e+11,8.393046e+10,1.141904e+11
6,a10ad10,0.000039,0.475925,20,0.000024,1.079281e+11,9.396793e+10,1.205203e+11
7,a18ad18,0.000031,0.557539,60,0.000024,1.276359e+11,1.137835e+11,1.425782e+11
8,a20ad20,0.000031,0.534847,60,0.000024,1.222818e+11,1.091525e+11,1.364452e+11
9,a21ad21,0.000031,0.536467,60,0.000024,1.226640e+11,1.094831e+11,1.368830e+11


In [6]:
# create pandas dataframe for temperatures
# open the json file
df_spectra_T = pd.DataFrame()
with open('../config_p1_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p1_off
    df_spectra_T = pd.concat([df_spectra_T, df])

with open('../config_p2_2302.json') as file:
    data = json.load(file)
    df = pd.read_json(json.dumps(data['T']), orient='index')
    df['p_off'] = p2_off
    df_spectra_T = pd.concat([df_spectra_T, df])
    
df_spectra_T = df_spectra_T.reset_index()
df_spectra_T = df_spectra_T.rename(columns={'index': 'id'})

folder_path = '../data/2302/2302spectr/'
extension = '.dat'

df_spectra_T['file'] = folder_path + df_spectra_T['id'] + extension
df_spectra_T['wavelength'] = 750.4
df_spectra_T['i_abs'] = df_spectra_T.apply(lambda x: functions.get_absolute_intensity_2(x['file'],x['t'], x['wavelength']), axis=1)
df_spectra_T.columns = ['id', 'p (mbar)', 't (s)', 'w (%)', 'p_off (mbar)', 'file',
                       'wavelength (nm)', 'i_abs ($cm^{-3} s^{-1}$)']
df_spectra_T


,id,p (mbar),t (s),w (%),p_off (mbar),file,wavelength (nm),i_abs ($cm^{-3} s^{-1}$)
0,a15ad15,0.000038,0.01,20,0.000024,../data/2302/2302spectr/a15ad15.dat,750.4,1.022433e+08
1,a16ad16,0.000038,0.01,20,0.000024,../data/2302/2302spectr/a16ad16.dat,750.4,1.066341e+08
2,a17ad17,0.000038,0.01,20,0.000024,../data/2302/2302spectr/a17ad17.dat,750.4,1.047524e+08
3,a23ad23,0.000031,0.01,60,0.000024,../data/2302/2302spectr/a23ad23.dat,750.4,2.320861e+08
4,a24ad24,0.000031,0.01,60,0.000024,../data/2302/2302spectr/a24ad24.dat,750.4,2.276953e+08
5,a25ad25,0.000031,0.01,60,0.000024,../data/2302/2302spectr/a25ad25.dat,750.4,2.245590e+08
6,a4ad4,0.000000,0.01,40,0.000024,../data/2302/2302spectr/a4ad4.dat,750.4,1.260792e+08
7,a5ad5,0.000000,0.01,40,0.000024,../data/2302/2302spectr/a5ad5.dat,750.4,1.254519e+08
8,a6ad6,0.000000,0.01,40,0.000024,../data/2302/2302spectr/a6ad6.dat,750.4,1.191793e+08
9,a31ad31,0.000147,0.01,20,0.000134,../data/2302/2302spectr/a31ad31.dat,750.4,3.236660e+08


In [7]:
# time to shoot myself
df = pd.merge(df_spectra_T, df_spectra_n, on=['w (%)', 'p_off (mbar)'])
#df['p_correction'] = 1/1.29
a = df.groupby(['p_off (mbar)', 'w (%)'], as_index=False).mean()
a['pec ($cm^{3}s^{-1}$)'] = a.apply(lambda x: functions.get_pec(x['i_abs ($cm^{-3} s^{-1}$)'], x['p_off (mbar)'], x['n ($cm^{-3}$)']), axis=1)
a

,p_off (mbar),w (%),p (mbar)_x,t (s),wavelength (nm),i_abs ($cm^{-3} s^{-1}$),p (mbar)_y,ratio,n ($cm^{-3}$),n_min ($cm^{-3}$),n_max ($cm^{-3}$),pec ($cm^{3}s^{-1}$)
0,0.000024,20,0.000038,0.01,750.4,1.045433e+08,0.000039,0.461178,1.035598e+11,8.764792e+10,1.165347e+11,1.729605e-14
1,0.000024,40,0.000000,0.01,750.4,1.235702e+08,0.000000,0.474157,1.074542e+11,9.321022e+10,1.200425e+11,1.970299e-14
2,0.000024,60,0.000031,0.01,750.4,2.281134e+08,0.000031,0.541394,1.238265e+11,1.104886e+11,1.382147e+11,3.156305e-14
3,0.000134,20,0.000147,0.01,750.4,3.182297e+08,0.000147,0.472599,1.069674e+11,9.254223e+10,1.196212e+11,9.129307e-15
4,0.000134,40,0.000155,0.01,750.4,5.133075e+08,0.000155,0.507000,1.157113e+11,1.034693e+11,1.289188e+11,1.361288e-14
5,0.000134,60,0.000156,0.01,750.4,5.731063e+08,0.000156,0.511087,1.166756e+11,1.043034e+11,1.300234e+11,1.507312e-14


In [8]:
# adas reading and interpolation
df_adas = functions.read_adas_file('../data/adas/ar_750.dat')
mesh_n, mesh_pec = np.meshgrid(np.log10(a['n ($cm^{-3}$)']), np.log10(a['pec ($cm^{3}s^{-1}$)']))
a['T'] = functions.interpolate_dataframe(df_adas, (np.log10(a['n ($cm^{-3}$)']), np.log10(a['pec ($cm^{3}s^{-1}$)'])))
df = a
df

,p_off (mbar),w (%),p (mbar)_x,t (s),wavelength (nm),i_abs ($cm^{-3} s^{-1}$),p (mbar)_y,ratio,n ($cm^{-3}$),n_min ($cm^{-3}$),n_max ($cm^{-3}$),pec ($cm^{3}s^{-1}$),T
0,0.000024,20,0.000038,0.01,750.4,1.045433e+08,0.000039,0.461178,1.035598e+11,8.764792e+10,1.165347e+11,1.729605e-14,3.151671
1,0.000024,40,0.000000,0.01,750.4,1.235702e+08,0.000000,0.474157,1.074542e+11,9.321022e+10,1.200425e+11,1.970299e-14,3.182733
2,0.000024,60,0.000031,0.01,750.4,2.281134e+08,0.000031,0.541394,1.238265e+11,1.104886e+11,1.382147e+11,3.156305e-14,3.295152
3,0.000134,20,0.000147,0.01,750.4,3.182297e+08,0.000147,0.472599,1.069674e+11,9.254223e+10,1.196212e+11,9.129307e-15,3.001345
4,0.000134,40,0.000155,0.01,750.4,5.133075e+08,0.000155,0.507000,1.157113e+11,1.034693e+11,1.289188e+11,1.361288e-14,3.096268
5,0.000134,60,0.000156,0.01,750.4,5.731063e+08,0.000156,0.511087,1.166756e+11,1.043034e+11,1.300234e+11,1.507312e-14,3.120366


In [9]:
# plotting data
fig_spectr, ax_spectr = plt.subplots(1,1)
ax_spectr.errorbar(w_em_p1, n_avg_p1, yerr=n_max_p1-n_min_p1, fmt='o', label='p = {} mbar'.format(p1_off))
ax_spectr.errorbar(w_em_p2, n_avg_p2, yerr=n_max_p2-n_min_p2, fmt='o', label='p = {} mbar'.format(p2_off))
ax_spectr.legend()
ax_spectr.grid()
# second pression spectra analysis

In [33]:
# langmuir density and profile temperature
#                        path to results data
df = agg.aggregate_data('../langmuir/data/data-2302')
# the grouped by columns will become indexes
a = df.groupby(['P_off', 'W_em'])['n_e', 'err_n_e', 'T_e', 'err_T_e'].mean()
# set plot stles, titles and figures. Need to add the labels though
fig = plt.figure()
ax_ne = fig.add_subplot(211)
ax_ne.set_title('$n_e$ $cm^{-3}$')
ax_ne.grid()
ax_te = fig.add_subplot(212)
ax_te.set_title('$T_e$ (eV)')
ax_te.grid()
plt.tight_layout()  # this is useful for overlapping figures

# difficult code: iterate over the values of the pressure
# indexes (there are just two values in this case) and
# take the values of the w, n, and T for each p.
# Then plot all the data with their labels
for p in a.index.get_level_values(0).unique():
    w = a.index.get_level_values(1).unique()
    n_e = a.loc[p,:]['n_e']
    err_n_e = a.loc[p,:]['err_n_e']
    T_e = a.loc[p,:]['T_e']
    err_T_e = a.loc[p,:]['err_T_e']
    ax_ne.errorbar(w, n_e, yerr=err_n_e, fmt='o', label="p: {} mbar".format(p))
    ax_te.errorbar(w, T_e, yerr=err_T_e, fmt='o', label="p: {} mbar".format(p))
    
# set legend for the plots
ax_ne.legend()
ax_te.legend()

Reading dataset:['2302lang10', '2302lang09', '2302lang03', '2302lang11', '2302lang04', '2302lang06', '2302lang05', '2302lang08', '2302lang02', '2302lang07', '2302lang12', '2302lang01']
